<a href="https://colab.research.google.com/github/ywangumichigan/EECS595-Project/blob/main/RAG_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet wikipedia
!pip install --quiet nltk
!pip install --quiet gensim
!pip install --quiet faiss-cpu
!pip install --quiet langchain-text-splitters
!pip install --quiet transformers
!pip install --quiet torch


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 17.2 MB/s eta 0:00:00


In [2]:
# --- Imports ---
import nltk
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Download tokenizer data ---
nltk.download('punkt')

# --- Step 1: Collect Wikipedia data ---


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# --- Step 2: Split long pages into smaller chunks ---
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
import langchain

print("LangChain version:", langchain.__version__)



LangChain version: 1.1.3


In [5]:
# --- Install dependencies ---
!pip install --quiet wikipedia langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers transformers

# --- Imports ---
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

# --- Step 1: Collect small Wikipedia sample ---
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
import wikipedia

topics = ["Algebra", "Calculus", "Derivative", "Integral",
          "Matrix (mathematics)", "Probability", "Statistics",
          "Geometry", "Trigonometry", "Number theory","Imaginary Number"]

def prune_wiki(text):
    stop_markers = [
        "== See also ==",
        "== References ==",
        "== External links ==",
        "== Further reading =="
    ]
    for marker in stop_markers:
        idx = text.find(marker)
        if idx != -1:
            text = text[:idx]
    return text

raw_docs = []
for topic in topics:
    try:
        page = wikipedia.page(topic)
        content = prune_wiki(page.content)
        raw_docs.append({"title": topic, "content": content})
        print("Collected:", topic)
    except:
        print("Skipped:", topic)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " ", ""],
)

docs = []
for d in raw_docs:
    chunks = splitter.create_documents(
        [d["content"]],
        metadatas=[{"title": d["title"]}]
    )
    docs.extend(chunks)

print("Total chunks:", len(docs))

wiki_corpus = []   # 👈 this is what later cells will use
for topic in topics:
    try:
        page = wikipedia.page(topic)
        content = prune_wiki(page.content)
        wiki_corpus.append({"title": topic, "content": content})
        print("Collected:", topic)
    except Exception as e:
        print("Skipped:", topic, "->", e)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
Collected: Algebra
Collected: Calculus


/usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Skipped: Derivative
Collected: Integral
Collected: Matrix (mathematics)
Collected: Probability
Collected: Statistics
Collected: Geometry
Collected: Trigonometry
Collected: Number theory
Collected: Imaginary Number
Total chunks: 1430
Collected: Algebra
Collected: Calculus
Skipped: Derivative -> "derivation" may refer to: 
Morphological derivation
Parse tree
Derivative work
Derivation proceeding
derived row
Derivation (differential  algebra)
Formal proof
Vilfredo Pareto
Derive (disambiguation)
Derivative
Derivative (disambiguation)
All pages with titles containing Derivation
Collected: Integral
Collected: Matrix (mathematics)
Collected: Probability
Collected: Statistics
Collected: Geometry
Collected: Trigonometry
Collected: Number theory
Collected: Imaginary Number


In [6]:
banned_sections = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Notes",
    "Bibliography"
]

latex_block_pattern = re.compile(
    r"\{\s*\\displaystyle[^}]*\}|\\begin\{aligned\}.*?\\end\{aligned\}",
    re.S,
)

def clean_wiki_page(text: str) -> str:
    lines = text.split("\n")
    cleaned = []
    stop = False
    for line in lines:
        if any(line.strip().startswith(f"== {sec}") for sec in banned_sections):
            stop = True
        if stop:
            continue
        line = re.sub(r"\[[0-9]+\]", "", line)
        if line.strip():
            cleaned.append(line)

    text = "\n".join(cleaned)

    text = latex_block_pattern.sub(" ", text)

    text = re.sub(r"\s+", " ", text)

    return text.strip()

wiki_corpus_clean = []
for item in wiki_corpus:
    wiki_corpus_clean.append({
        "title": item.get("title", ""),
        "content": clean_wiki_page(item["content"])
    })

In [7]:


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

texts = [item["content"] for item in wiki_corpus_clean]
metadatas = [{"title": item["title"]} for item in wiki_corpus_clean]

docs = splitter.create_documents(texts, metadatas=metadatas)
print("Total chunks:", len(docs))

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3,"fetch_k": 20, "lambda_mult": 0.7}
)



Total chunks: 819


/tmp/ipython-input-460957146.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from google.colab import drive
drive.mount("/content/drive")
!pip install -q transformers langchain-community accelerate safetensors



Mounted at /content/drive


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

model_dir = "/content/drive/MyDrive/model"

base_model_id = "Qwen/Qwen3-1.7B-base"

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.eos_token_id

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.4,
    top_p=0.9,
    do_sample=False,
)

llm = HuggingFacePipeline(pipeline=gen_pipe)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-4237655457.py:35: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen_pipe)


In [10]:
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
#from langchain_community.llms import HuggingFacePipeline  # make sure this import is from langchain_community

## --- Step 4: Load small question-answering model ---
#model_id = "google/flan-t5-large"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
##model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
#pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)

#llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["context", "question"],
  template = (
    "You are a helpful math tutor.\n\n"
    "You will be given some context passages and a question.\n"
    "Use the context only if it directly helps; if not, ignore it and solve the problem yourself.\n"
    "Think through the problem step by step internally, but DO NOT show your reasoning.\n"
    "Your output must follow these rules:\n"
    "- Output ONLY ONE line in total.\n"
    "- That line must be a LaTeX box, for example: \\boxed{{2}} or \\boxed{{-\\tfrac{{5}}{{3}}}}.\n"
    "- Replace the number in the box with the correct final answer.\n"
    "- Do NOT literally write the characters '<' or '>' in your answer.\n\n"
    "Context:\n{context}\n\n"
    "Question:\n{question}\n\n"
    "Answer:"
) )

def format_docs(docs):
    # docs is a list of Document objects from your retriever
    return "\n\n".join(doc.page_content for doc in docs)

# RAG chain: question (str) -> answer (str)
qa_chain = (
    {
        "context": retriever | format_docs,   # retrieve docs then format
        "question": RunnablePassthrough(),    # pass the original question through
    }
    | prompt                                  # fill the PromptTemplate
    | llm                                     # your HuggingFacePipeline LLM
    | StrOutputParser()                       # get back a plain string
)

question = "How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?"
answer = qa_chain.invoke(question)
print(answer)



You are a helpful math tutor.

You will be given some context passages and a question.
Use the context only if it directly helps; if not, ignore it and solve the problem yourself.
Think through the problem step by step internally, but DO NOT show your reasoning.
Your output must follow these rules:
- Output ONLY ONE line in total.
- That line must be a LaTeX box, for example: \boxed{2} or \boxed{-\tfrac{5}{3}}.
- Replace the number in the box with the correct final answer.
- Do NOT literally write the characters '<' or '>' in your answer.

Context:
t y . }}}={\sqrt {(-x)\cdot (-y)}}\mathrel {\stackrel {\text{ (fallacy) }}{=}} {\sqrt {-x{\vphantom {ty}}}}\cdot {\sqrt {-y{\vphantom {ty}}}}=i{\sqrt {x{\vphantom {ty}}}}\cdot i{\sqrt {y{\vphantom {ty}}}}=-{\sqrt {x\cdot y{\vphantom {ty}}}}\,.} But the result is clearly nonsense. The step where the square root was broken apart was illegitimate. (See Mathematical fallacy.)

one asks whether there are any rational points (points all of whose c

In [12]:
import re
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
import torch

print("Loading Hendrycks Math (Algebra)...")
ds_algebra = load_dataset("EleutherAI/hendrycks_math", "algebra")

Loading Hendrycks Math (Algebra)...


README.md: 0.00B [00:00, ?B/s]

algebra/train-00000-of-00001.parquet:   0%|          | 0.00/505k [00:00<?, ?B/s]

algebra/test-00000-of-00001.parquet:   0%|          | 0.00/353k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1744 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1187 [00:00<?, ? examples/s]

In [13]:

def extract_last_boxed(text: str):
    """
    Extract the content inside the *last* \boxed{...} in `text`,
    correctly handling nested braces like \frac{a}{b}.
    Returns the inner LaTeX like '\\frac{9}{5}' or None if not found.
    """
    marker = r"\boxed{"
    idx = text.rfind(marker)
    if idx == -1:
        return None

    i = idx + len(marker)
    depth = 0
    content_chars = []

    while i < len(text):
        ch = text[i]
        if ch == "{":
            depth += 1
            content_chars.append(ch)
        elif ch == "}":
            if depth == 0:
                # this is the closing brace that matches \boxed{
                break
            else:
                depth -= 1
                content_chars.append(ch)
        else:
            content_chars.append(ch)
        i += 1

    content = "".join(content_chars).strip()
    return content if content else None


def extract_all_boxed(text):
    results = []
    i = 0
    key = r"\boxed"

    while True:
        start = text.find(key, i)
        if start == -1:
            break

        j = start + len(key)
        while j < len(text) and text[j].isspace():
            j += 1

        if j >= len(text) or text[j] != '{':
            i = start + 1
            continue

        depth = 0
        content_start = j + 1

        for k in range(content_start, len(text)):
            if text[k] == '{':
                depth += 1
            elif text[k] == '}':
                if depth == 0:
                    results.append(text[content_start:k].strip())
                    i = k + 1
                    break
                depth -= 1
        else:
            break

    return results


def extract_ref_answer(ref_text):
    return extract_last_boxed(ref_text)

def normalize_ans(ans):
    if ans is None:
        return None
    ans = ans.replace(" ", "").replace("\\frac", "frac").replace("\\dfrac", "frac")
    return ans




In [14]:
def exact_match(preds, refs):
    """
    preds: list of model predictions *already extracted from \\boxed{...}*
    refs:  list of reference solution strings (full LaTeX solutions from dataset)

    Returns: exact-match accuracy on examples where we have a non-Null prediction.
    """
    correct = 0
    total = 0

    for pred, ref in zip(preds, refs):
        ref_box = extract_ref_answer(ref)      # uses extract_last_boxed internally

        pred_norm = normalize_ans(pred)
        ref_norm = normalize_ans(ref_box)

        if pred_norm is None or pred_norm == "Null":
            continue

        total += 1
        if pred_norm == ref_norm:
            correct += 1

    return correct / max(total, 1)


In [15]:
def answer_with_rag(question: str):

    answer_text = qa_chain.invoke(question)
    return answer_text, None



In [16]:
def generate_rag_answer(problem, max_new_tokens=8192):

    answer, ctx_docs = answer_with_rag(problem)
    full_output = f"RAG ANSWER:\n{answer}"
    box = extract_last_boxed(answer)

    if box is None or box.strip() == "":
        return "Null", full_output

    return box.strip(), full_output


In [17]:
def run_rag_eval(n=20, split="test", show_debug=False):

    preds = []
    refs = []
    outputs = []

    for i in tqdm(range(n)):
        item = ds_algebra[split][i]
        problem = item["problem"]
        reference = item["solution"]

        pred, full_output = generate_rag_answer(problem)

        preds.append(pred)
        refs.append(reference)
        outputs.append(full_output)

        if show_debug:
            print("\n=====================")
            print("Problem:", problem)
            # print("\nReference (GT):", reference)
            # print("\nModel output:\n", full_output)
            print("Predicted boxed:", pred)
            print("=====================\n")

    acc = exact_match(preds, refs)
    print(f"\n📌 RAG Exact Match Accuracy on {n} problems = {acc:.4f}")
    return acc, preds, refs, outputs



In [18]:
'''
def build_rag_prompt(question: str, context_docs) -> str:
    """
    Build a RAG prompt with a strict 'helpful math tutor' instruction.
    """

    # Merge retrieved chunks (keep titles if you like)
    context_blocks = []
    for i, doc in enumerate(context_docs):
        context_blocks.append(
            f"[Source {i+1} — {doc.metadata.get('title', 'Wikipedia')}]\n{doc.page_content}"
        )
    context_str = "\n\n".join(context_blocks)

    # Base template
    template = (
    "You are a helpful math tutor.\n\n"
    "You will be given some context passages and a question.\n"
    "Use the context only if it directly helps; if not, ignore it and solve the problem yourself.\n"
    "Think through the problem step by step internally, but DO NOT show your reasoning.\n"
    "Your output must follow these rules:\n"
    "- Output ONLY ONE line in total.\n"
    "- That line must be a LaTeX box, for example: \\boxed{{2}} or \\boxed{{-\\tfrac{{5}}{{3}}}}.\n"
    "- Replace the number in the box with the correct final answer.\n"
    "- Do NOT literally write the characters '<' or '>' in your answer.\n\n"
    "Context:\n{context}\n\n"
    "Question:\n{question}\n\n"
    "Answer:"
)
    # Fill in context + question
    return template.format(context=context_str, question=question)
'''

'\ndef build_rag_prompt(question: str, context_docs) -> str:\n    """\n    Build a RAG prompt with a strict \'helpful math tutor\' instruction.\n    """\n\n    # Merge retrieved chunks (keep titles if you like)\n    context_blocks = []\n    for i, doc in enumerate(context_docs):\n        context_blocks.append(\n            f"[Source {i+1} — {doc.metadata.get(\'title\', \'Wikipedia\')}]\n{doc.page_content}"\n        )\n    context_str = "\n\n".join(context_blocks)\n\n    # Base template\n    template = (\n    "You are a helpful math tutor.\n\n"\n    "You will be given some context passages and a question.\n"\n    "Use the context only if it directly helps; if not, ignore it and solve the problem yourself.\n"\n    "Think through the problem step by step internally, but DO NOT show your reasoning.\n"\n    "Your output must follow these rules:\n"\n    "- Output ONLY ONE line in total.\n"\n    "- That line must be a LaTeX box, for example: \\boxed{{2}} or \\boxed{{-\\tfrac{{5}}{{3}}}}.\n"\n

In [19]:

acc, preds, refs, outputs = run_rag_eval(50, show_debug=True)


  2%|▏         | 1/50 [00:01<01:15,  1.55s/it]


Problem: How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?
Predicted boxed: 2



  4%|▍         | 2/50 [00:03<01:14,  1.54s/it]


Problem: What is the positive difference between $120\%$ of 30 and $130\%$ of 20?
Predicted boxed: 14



  6%|▌         | 3/50 [00:05<01:29,  1.90s/it]

...


Problem: Simplify $(2-2i)(5+5i)$, where $i^2 = -1.$
Predicted boxed: 10+10i



 98%|█████████▊| 49/50 [02:44<00:00,  1.12it/s]


Problem: Factor the following expression: $7x^2-63$.
Predicted boxed: 7(x^2-9)



100%|██████████| 50/50 [02:45<00:00,  3.31s/it]


Problem: Let $f(y) = y^4 -3y^3 +y - 3$ and $g(y) = y^3 + 7y^2 -2$. Find $f(y) + g(y)$.  Write your answer as a polynomial with terms of decreasing degree.
Predicted boxed: y^4 + y^3 + 7y^2 -2


📌 RAG Exact Match Accuracy on 50 problems = 0.1200


In [20]:
import json
from collections import Counter

json_path = model_dir = "/content/drive/MyDrive/test_math.json"

with open(json_path, "r", encoding="utf-8") as f:
    all_items = json.load(f)

print("总题数：", len(all_items))
print("level 分布：", Counter([x["level"] for x in all_items]))


总题数： 5000
level 分布： Counter({'Level 5': 1324, 'Level 4': 1214, 'Level 3': 1131, 'Level 2': 894, 'Level 1': 437})


In [21]:
from tqdm import tqdm

def run_rag_eval_level(level: str, n: int | None = None, show_debug: bool = False):

    items = [x for x in all_items if x["level"] == level]

    if n is not None:
        items = items[:n]

    preds = []
    refs = []
    outputs = []

    for item in tqdm(items):
        problem = item["problem"]
        reference = item["output"]

        pred, full_output = generate_rag_answer(problem)

        preds.append(pred)
        refs.append(reference)
        outputs.append(full_output)

        if show_debug:
            print("\n=====================")
            print("Problem:", problem)
            print("Predicted boxed:", pred)
            print("=====================\n")

    acc = exact_match(preds, refs)
    print(f"\n📌 {level}: Exact Match Accuracy on {len(items)} problems = {acc:.4f}")
    return acc, preds, refs, outputs


In [22]:
levels = ["Level 1", "Level 2", "Level 3", "Level 4", "Level 5"]

results_by_level = {}
for lv in levels:
    print(f"\n========== Evaluating {lv} ==========")
    acc, preds, refs, outs = run_rag_eval_level(lv, n=50, show_debug=False)
    results_by_level[lv] = acc

print(results_by_level)



========== Evaluating Level 1 ==========


100%|██████████| 50/50 [00:32<00:00,  1.56it/s]



📌 Level 1: Exact Match Accuracy on 50 problems = 0.3600

========== Evaluating Level 2 ==========


100%|██████████| 50/50 [00:47<00:00,  1.06it/s]



📌 Level 2: Exact Match Accuracy on 50 problems = 0.3200

========== Evaluating Level 3 ==========


...


📌 Level 3: Exact Match Accuracy on 50 problems = 0.1400

========== Evaluating Level 4 ==========


100%|██████████| 50/50 [01:02<00:00,  1.26s/it]



📌 Level 4: Exact Match Accuracy on 50 problems = 0.0600

========== Evaluating Level 5 ==========


100%|██████████| 50/50 [04:19<00:00,  5.20s/it]


📌 Level 5: Exact Match Accuracy on 50 problems = 0.0200
{'Level 1': 0.36, 'Level 2': 0.32, 'Level 3': 0.14, 'Level 4': 0.06, 'Level 5': 0.02}
